In [1]:
import numpy as np 
import pandas as pd
import sys
import os

from ase.atoms import Atom, Atoms
from ase.parallel import paropen

import seaborn as sns
sns.set()

In [2]:
def get_coulombmatrix(molecule, largest_mol_size=None):
    """
    This function generates a coulomb matrix for the given molecule
    if largest_mol size is provided matrix will have dimension lm x lm.
    Padding is provided for the bottom and right _|
    """
    numberAtoms = len(molecule.atoms)
    if largest_mol_size == None or largest_mol_size == 0: largest_mol_size = numberAtoms

    cij = np.zeros((largest_mol_size, largest_mol_size))

    xyzmatrix = [[atom.position.x, atom.position.y, atom.position.z] for atom in molecule.atoms]
    chargearray = [atom.atomic_number for atom in molecule.atoms]

    for i in range(numberAtoms):
        for j in range(numberAtoms):
            if i == j:
                cij[i][j] = 0.5 * chargearray[i] ** 2.4  # Diagonal term described by Potential energy of isolated atom
            else:
                dist = np.linalg.norm(np.array(xyzmatrix[i]) - np.array(xyzmatrix[j]))
                cij[i][j] = chargearray[i] * chargearray[j] / dist  # Pair-wise repulsion
    return cij

In [4]:
from dscribe.descriptors import CoulombMatrix

atomic_numbers = [1, 8]
rcut = 6.0
nmax = 8
lmax = 6

# Setting up the CM descriptor
cm = CoulombMatrix(
    n_atoms_max=6,
)

# Creation
from ase.build import molecule

# Molecule created as an ASE.Atoms
methanol = molecule("CH3OH")

# Create CM output for the system
cm_methanol = cm.create(methanol)

print(cm_methanol)
print("flattened", cm_methanol.shape)
print()
# Create output for multiple system
samples = [molecule("H2O"), molecule("NO2"), molecule("CO2")]
coulomb_matrices = cm.create(samples)            # Serial
coulomb_matrices = cm.create(samples, n_jobs=2)  # Parallel

# No flattening
cm = CoulombMatrix(
    n_atoms_max=6, flatten=False
)
cm_methanol = cm.create(methanol)

print(cm_methanol)
print("not flattened", cm_methanol.shape)
print()


# Zero-padding
cm = CoulombMatrix(
    n_atoms_max=10, flatten=False
)
cm_methanol = cm.create(methanol)

print("zero-padded", cm_methanol)
print(cm_methanol.shape)
print()

# Not meant for periodic systems
methanol.set_pbc([True, True, True])
methanol.set_cell([[10.0, 0.0, 0.0],
    [0.0, 10.0, 0.0],
    [0.0, 0.0, 10.0],
    ])

cm = CoulombMatrix(
    n_atoms_max=6, flatten=False
)

cm_methanol = cm.create(methanol)
print("with pbc", cm_methanol)
print()

# Invariance
cm = CoulombMatrix(
    n_atoms_max=6,
    flatten=False,
    permutation="sorted_l2"
)

# Translation
methanol.translate((5, 7, 9))
cm_methanol = cm.create(methanol)
print(cm_methanol)
print()

# Rotation
methanol.rotate(90, 'z', center=(0, 0, 0))
cm_methanol = cm.create(methanol)
print(cm_methanol)
print()

# Permutation
upside_down_methanol = methanol[::-1]
cm_methanol = cm.create(upside_down_methanol)
print(cm_methanol)
print()

# Options for permutation

# No sorting
cm = CoulombMatrix(
    n_atoms_max=6, flatten=False,
    permutation='none'
)

cm_methanol = cm.create(methanol)
print(methanol.get_chemical_symbols())
print("in order of appearance", cm_methanol)
print()

# Sort by Euclidean (L2) norm.
cm = CoulombMatrix(
    n_atoms_max=6, flatten=False,
    permutation='sorted_l2'
)

cm_methanol = cm.create(methanol)
print("default: sorted by L2-norm", cm_methanol)
print()

# Random
cm = CoulombMatrix(
    n_atoms_max=6, flatten=False,
    permutation='random',
    sigma=70,
    seed=None
)

cm_methanol = cm.create(methanol)
print("randomly sorted", cm_methanol)
print()

# Eigenspectrum
cm = CoulombMatrix( 
    n_atoms_max=6, flatten=False,
    permutation='eigenspectrum'
)

cm_methanol = cm.create(methanol)
print("eigenvalues", cm_methanol)
print()


[[73.51669472 33.73297539  8.24741496  3.96011613  3.808905    3.808905
  33.73297539 36.8581052   3.08170819  5.50690929  5.47078813  5.47078813
   8.24741496  3.08170819  0.5         0.35443545  0.42555057  0.42555057
   3.96011613  5.50690929  0.35443545  0.5         0.56354208  0.56354208
   3.808905    5.47078813  0.42555057  0.56354208  0.5         0.56068143
   3.808905    5.47078813  0.42555057  0.56354208  0.56068143  0.5       ]]
flattened (1, 36)

[[73.51669472 33.73297539  8.24741496  3.96011613  3.808905    3.808905  ]
 [33.73297539 36.8581052   3.08170819  5.50690929  5.47078813  5.47078813]
 [ 8.24741496  3.08170819  0.5         0.35443545  0.42555057  0.42555057]
 [ 3.96011613  5.50690929  0.35443545  0.5         0.56354208  0.56354208]
 [ 3.808905    5.47078813  0.42555057  0.56354208  0.5         0.56068143]
 [ 3.808905    5.47078813  0.42555057  0.56354208  0.56068143  0.5       ]]
not flattened (6, 6)

zero-padded [[73.51669472 33.73297539  8.24741496  3.96011613  3